In [1]:
%load_ext autoreload  
%autoreload 2 

from subset_active_learning.active_learning.subset_classifier import get_df_from_db
import os
import sqlite3
import json
import seaborn as sns
from tqdm.notebook import tqdm

import torch
import numpy as np
import datasets
import transformers
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DataCollator

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = get_df_from_db("./sst_results.db")

In [4]:
seed = 0
annealing_runs = 5000
total_sample_size = 1000
n_search = 100
db_name = 'sst_results'
anneal_factor = 0.1

model_card = "google/electra-small-discriminator"
pretraining = True
max_steps = 6000
eval_steps = 300
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_card)

In [ ]:
sst2 = datasets.load_dataset('sst')

In [7]:
sst2

NameError: name 'sst2' is not defined

In [8]:
'''
def tokenize_function(examples, field='sentence'):
    return tokenizer(examples[field], padding=False, truncation=True)
tokenized_sst2 = sst2.map(tokenize_function, batched=True)
lengths = [len(i) for i in tokenized_sst2['train']['input_ids']]
max_length = np.quantile(lengths, 0.9)
max_length = np.max(lengths)
'''
max_length = 66
max_length

66

In [9]:
def tokenize_function(examples, field='sentence'):
    return tokenizer(examples[field], padding='max_length', max_length=max_length, truncation=True)

tokenized_sst2 = sst2.map(tokenize_function, batched=False)

NameError: name 'sst2' is not defined

In [ ]:
# binarize label
tokenized_sst2 = tokenized_sst2.rename_column('label', 'scalar_label')
tokenized_sst2 = tokenized_sst2.map(lambda x: {'labels' : 0 if x['scalar_label'] < 0.5 else 1})

print(tokenized_sst2['test'][-10:]['sentence'])
print(tokenized_sst2['test'][-10:]['labels'])

In [ ]:
tokenized_sst2.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
sst2_downsample = tokenized_sst2['train'].shuffle(seed=seed).select(range(0, total_sample_size))

In [ ]:
val_dataloader = torch.utils.data.DataLoader(tokenized_sst2['validation'], shuffle=False, batch_size=batch_size, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(tokenized_sst2['test'], shuffle=False, batch_size=batch_size, pin_memory=True)

In [ ]:
def create_hf(model_card, num_labels=2):
    tokenizer = AutoTokenizer.from_pretrained(model_card)
    model = AutoModelForSequenceClassification.from_pretrained(model_card, num_labels=num_labels)
    return model, tokenizer

In [ ]:
metric = datasets.load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
with sqlite3.connect("./sst_results.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM states ORDER BY objective DESC LIMIT 1 OFFSET %d" % 2)
    r = cursor.fetchone()
    print(r)

In [ ]:
test_nth_best = np.array(json.loads(r[0]))

In [ ]:
test_nth_best

In [ ]:
total_idx = np.arange(total_sample_size)

In [14]:
test_nth_best = np.array(
    [
        517,
        882,
        82,
        341,
        477,
        381,
        611,
        377,
        837,
        217,
        712,
        216,
        514,
        728,
        523,
        842,
        614,
        316,
        7,
        536,
        540,
        636,
        766,
        479,
        461,
        85,
        852,
        270,
        460,
        3,
        664,
        56,
        605,
        315,
        741,
        352,
        175,
        112,
        176,
        120,
        349,
        485,
        8,
        459,
        146,
        627,
        159,
        325,
        796,
        888,
        318,
        430,
        364,
        467,
        510,
        704,
        895,
        496,
        812,
        727,
        797,
        184,
        744,
        322,
        22,
        669,
        917,
        103,
        548,
        631,
        577,
        137,
        194,
        111,
        81,
        321,
        66,
        228,
        145,
        63,
        602,
        24,
        233,
        511,
        752,
        681,
        589,
        33,
        791,
        6,
        731,
        231,
        114,
        101,
        422,
        735,
        382,
        23,
        88,
        901,
    ]
)

# np.random.seed(42)
# random.randint(0, len(test_nth_best) - 1)

# create_new_subset_in_place(base_subset=test_nth_best)
# len(set(test_nth_best))

In [16]:
type(test_nth_best)

numpy.ndarray

In [255]:
create_new_subset_in_place(base_subset=test_nth_best)

swapping out: 123 at index 51 | swapping in: 124


In [229]:
np.random.seed(0)
print(np.random.randint(0, 5))

4


In [92]:
len(set(test_nth_best))

100

In [149]:
len(set(np.random.choice(total_sample_size, size=n_search, replace=False)))

100

In [273]:
%timeit create_new_subset_in_place(test_nth_best)

72.4 µs ± 3.89 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [264]:
import random
def transition(idx):
    total_idx = np.arange(total_sample_size)
    not_in = np.setdiff1d(total_idx, idx)
    a = np.random.choice(not_in)
    b = random.randint(0, len(idx) - 1)
    new_idx = idx.copy()
    new_idx[b] = a
    return new_idx

In [274]:
%timeit transition(test_nth_best)

75.7 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
np.setdiff1d(total_idx)

In [ ]:
random_idx = np.random.choice(total_sample_size, size=n_search)

In [ ]:
random_idx

In [ ]:
db_name = "test"

In [ ]:
try:
    con = sqlite3.connect('%s.db' % db_name)
    print(f"con: {con}")
    cur = con.cursor()
    cur.execute('''CREATE TABLE states
                   (indexes text, objective real)''')
    cur.execute('''CREATE INDEX idx_objective 
                    ON states (objective);''')
    
    # start from a random sample
    random_idx = np.random.choice(total_sample_size, size=n_search, replace=False)
    if (num_unique_samples := len(set(random_idx))) != total_sample_size:
        raise ValueError(f"Unexpected number of indices are selected. Expected {total_sample_size}, got {num_unique_samples}")
    print("random_idx: ", random_idx)
    cur.execute("INSERT INTO states VALUES ('%s', 0)" % json.dumps(random_idx.tolist()))
except Exception as e:
    print(e)
    pass
finally:
    con.commit()
    con.close()

In [ ]:
random_idx

In [10]:
def get_nth_best_subset(n):
    """ Select a single example -- the nth best by test accuracy to swap out
    """
    try:
        con = sqlite3.connect('%s.db' % db_name)
        cur = con.cursor()
        cur.execute("SELECT * FROM states ORDER BY objective DESC LIMIT 1 OFFSET %d" % n)
        r = cur.fetchone()
    finally:
        con.close()
    return (np.array(json.loads(r[0])), r[1])

def get_num_runs():
    try:
        con = sqlite3.connect('%s.db' % db_name)
        cur = con.cursor()
        cur.execute("SELECT COUNT(objective) FROM states")
        r = cur.fetchone()[0]
        con.commit()
    finally:
        con.close()
    return r

def insert_run(idx, obj):
    try:
        con = sqlite3.connect('%s.db' % db_name)
        cur = con.cursor()
        cur.execute("INSERT INTO states VALUES ('%s', %.8f)" % (json.dumps(idx.tolist()), obj))
        con.commit()
    finally:
        con.close()

In [3]:
from subset_active_learning.subset_selection import select

/Users/garylai/Dev/subset-active-learning/subset_active_learning/subset_selection/select.py:10: RuntimeWarning: fields may not start with an underscore, ignoring "_n_gpu"
  class SubsetSelector(BaseModel, extra=Extra.allow):


In [5]:
selector = select.SubsetSelector()

/Users/garylai/Dev/subset-active-learning/subset_active_learning/subset_selection/select.py:10: RuntimeWarning: fields may not start with an underscore, ignoring "_n_gpu"
  class SubsetSelector(BaseModel, extra=Extra.allow):
[autoreload of subset_active_learning.subset_selection.select failed: Traceback (most recent call last):
  File "/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 345, in update_cla

In [7]:
selector.db_name

'sst_results'

In [2]:
selector = select.SubsetSelector()

NameError: name 'select' is not defined

In [28]:
selector.db_name

'sst_results'

In [11]:
%timeit get_num_runs()

666 µs ± 80.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [268]:
def create_new_subset_in_place(base_subset: np.ndarray) -> None:
    """Create a new subset by swapping out one sample from the base_subset
    and swapping in a new sample. This is done inplace for efficiency
    """
    all_indices = np.arange(total_sample_size)
    available_examples = np.setdiff1d(all_indices, base_subset)
    # np.random.seed(42)
    in_sample = np.random.choice(available_examples)
    out_sample_idx = np.random.randint(0, len(base_subset) - 1)
    # print(f"swapping out: {base_subset[out_sample_idx]} at index {out_sample_idx} | swapping in: {in_sample}")
    base_subset[out_sample_idx] = in_sample

In [ ]:
def test_transition():
    pass

In [ ]:
def evaluate(model, val_dataloader):
    model.eval()
    val_pbar = tqdm(total=len(val_dataloader))
    for batch in val_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
        val_pbar.update(1)

    eval_dict = metric.compute()
    val_pbar.set_description('Acc: %.2f' % eval_dict['accuracy'])
    
    return eval_dict

In [ ]:
def train(model, train_dataset, tolerance=2):
    steps = 0
    epochs = 0
    best_acc = None
    patience = 0
    pbar = tqdm(total=max_steps)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)
    it = iter(train_dataloader)

    optimizer = torch.optim.AdamW(params=model.parameters(), lr=learning_rate, betas=(adam_beta1, adam_beta2), eps=adam_epsilon, weight_decay=weight_decay)
    lr_scheduler = transformers.get_scheduler("linear", optimizer=optimizer, num_warmup_steps=warmup_ratio*max_steps, num_training_steps=max_steps)

    while steps < max_steps:
        # training
        model.train()
        total_loss = 0.

        try:
            batch = next(it)
        except:
            epochs += 1
            it = iter(train_dataloader)
            batch = next(it)

        steps += 1
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        total_loss += loss.cpu()

        wandb.log({'loss' : loss})

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        pbar.set_description('Epoch: %d, Avg batch loss: %.2f' % (epochs, total_loss / steps))
        pbar.update(1)

        if steps % eval_steps == 0:
            model.eval()
            eval_dict = evaluate(model, val_dataloader)
            wandb.log({'sst:val_acc' : eval_dict['accuracy']})
            
            # early stopping
            if not best_acc or eval_dict['accuracy'] > best_acc:
                best_acc = eval_dict['accuracy']
            else:
                patience += 1
            
            if patience >= tolerance:
                break

In [ ]:
def one_run():
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # controls transition sampling
    # random.seed(seed)
    # np.random.seed(seed)

    i = get_num_runs()
    ratio = (annealing_runs - i) / annealing_runs if i < annealing_runs else 0
    idx = np.random.randint(0, int(ratio * i)) # 0 < idx < ratio * current_num_run; the bigger the n, the more exploration. The smaller the n, the greedier. 
    nth_best_index, quality = get_nth_best_subset(idx) # get the nth best subset (size 100)
    
    new_idx = create_new_subset_in_place(nth_best_index)
    train_dataset = sst2_downsample.select(new_idx) # you're always selecting `n_search` 100 datapoints
    assert(len(train_dataset) == n_search)
    
    model, tokenizer = create_hf(model_card, num_labels=2)
    model.to(device)
    
    wandb.init(project="simulated_annealing-sst", entity="johntzwei", tags=[model_card])
    wandb.log({'n_downsample' : total_sample_size})
    wandb.log({'n_search' : n_search})
    wandb.log({'model_card' : model_card})
    wandb.log({'indexes' : json.dumps(idx.tolist())})

    train(model, train_dataset)
    eval_dict = evaluate(model, test_dataloader)
    eval_dict = {'sst2_test:%s' % k : v for k, v in eval_dict.items()}
    new_quality = eval_dict['sst2_test:accuracy']
    wandb.log(eval_dict)
    
    insert_run(new_idx, new_quality)

In [ ]:
while True:
    one_run()